In [12]:
import numpy as np

def get_dummies(df, categoricalNumericFeatures=[]):
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]) or categoricalNumericFeatures.count(col) > 0:
            df = pd.get_dummies(df, columns=[col], prefix=col, drop_first=True)
    return df

def bin_groups(df, categoricalNumericFeatures=[], percent=.05):
    import pandas as pd

    for col in df:
        # if the column is categorical
        if not pd.api.types.is_numeric_dtype(df[col]) or categoricalNumericFeatures.count(col) > 0:
            for group, count in df[col].value_counts().iteritems():
                # For each categorical value, if the percentage of the column that contains that value is below the threshold, relabel to other
                if count / len(df) < percent:
                    df.loc[df[col] == group, col] = 'Other'
    return df

def drop_cols_too_unique(df, percent = .80):

    for col in df:
        pct = len(df[df[col] == 'Other'][col]) / len(df)
        if pct >=.80:
            df.drop(columns=col, inplace=True)
            print("dropped " + col + ", " + str(pct * 100) + "% unique")
    return df        

  

In [13]:
import pandas as pd

df_train = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\train.csv")
df_test = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\test.csv")

df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# sys.path.append(".")
from myFunctions import MissingData

df = df_train.copy()

categoricalNumericFeatures = ['Pclass','Survived'] # pclass is numeric but actually is a categorical feature
missingData = myFunctions.MissingData(df, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.8)
missingData.missing_values_report()

NameError: name 'myFunctions' is not defined

### Cabin and Embarked are categorical columns, and right now I will only be using mode imputation for categorical columns. Since imputing mode on cabin will likely not yield any great results, I will drop cabin and continue with imputation

In [ ]:
df.drop(columns='Cabin', inplace=True)
df = missingData.fillMissingData()

imputed 1 categorical column(s) using most_frequent.
imputed 1 numeric column(s) using mean.
Missing data check complete. No columns have missing data.


In [ ]:
# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.

df = bin_groups(df,categoricalNumericFeatures, percent=.05)
df = drop_cols_too_unique(df,percent=.8)
df.head()

dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


,Sex,Embarked,Pclass,Survived,PassengerId,Age,SibSp,Parch,Fare
0,male,S,3,0,1.0,22.0,1.0,0.0,7.2500
1,female,C,1,1,2.0,38.0,1.0,0.0,71.2833
2,female,S,3,1,3.0,26.0,0.0,0.0,7.9250
3,female,S,1,1,4.0,35.0,1.0,0.0,53.1000
4,male,S,3,0,5.0,35.0,0.0,0.0,8.0500


In [ ]:
# dummy code
df = get_dummies(df,categoricalNumericFeatures)
df.head()

,PassengerId,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Survived_1
0,1.0,22.0,1.0,0.0,7.2500,1,0,1,0,1,0
1,2.0,38.0,1.0,0.0,71.2833,0,0,0,0,0,1
2,3.0,26.0,0.0,0.0,7.9250,0,0,1,0,1,1
3,4.0,35.0,1.0,0.0,53.1000,0,0,1,0,0,1
4,5.0,35.0,0.0,0.0,8.0500,1,0,1,0,1,0


In [ ]:
from sklearn import preprocessing

dfMinMax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df),columns=df.columns)
dfMinMax.drop(columns='PassengerId', inplace=True)
#dfMinMax.hist(figsize=(15,10))

In [ ]:
X = dfMinMax.drop(columns='Survived_1')
y = dfMinMax['Survived_1']

#### Here is the estimated percent of the test set that will be classified correctly. In this case I don't really use it for anything since I just use logistic regression anyway.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from numpy import mean


cv = KFold(n_splits=10, random_state=12345, shuffle=True)

model_log = LogisticRegression(max_iter=100)
pctCorrect =  mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
print(pctCorrect)

0.8024843945068664


In [ ]:
model_log.fit(X,y)

LogisticRegression()

In [ ]:
X.head()

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,0.271174,0.125,0.0,0.014151,1.0,0.0,1.0,0.0,1.0
1,0.472229,0.125,0.0,0.139136,0.0,0.0,0.0,0.0,0.0
2,0.321438,0.000,0.0,0.015469,0.0,0.0,1.0,0.0,1.0
3,0.434531,0.125,0.0,0.103644,0.0,0.0,1.0,0.0,0.0
4,0.434531,0.000,0.0,0.015713,1.0,0.0,1.0,0.0,1.0


In [ ]:
passId = df_test['PassengerId']
df_test.drop(columns='PassengerId', inplace=True)

In [ ]:
missingData = MissingData(df_test, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.8)
missingData.missing_values_report()

Pclass	0.0%
Name	0.0%
Sex	0.0%
Age	20.57%
SibSp	0.0%
Parch	0.0%
Ticket	0.0%
Fare	0.24%
Cabin	78.23%
Embarked	0.0%


In [ ]:
df_test.drop(columns='Cabin', inplace=True)
df_test = missingData.fillMissingData()

imputed 2 numeric column(s) using mean.
Missing data check complete. No columns have missing data.


In [ ]:
# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.

df_test = bin_groups(df_test,categoricalNumericFeatures, percent=.05)
df_test = drop_cols_too_unique(df_test,percent=.8)
df_test.head()

dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


,Pclass,Sex,Embarked,Age,SibSp,Parch,Fare
0,3,male,Q,34.5,0.0,0.0,7.8292
1,3,female,S,47.0,1.0,0.0,7.0000
2,2,male,Q,62.0,0.0,0.0,9.6875
3,3,male,S,27.0,0.0,0.0,8.6625
4,3,female,S,22.0,1.0,1.0,12.2875


In [ ]:
df_test = get_dummies(df_test,categoricalNumericFeatures)
df_test.head()

,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,34.5,0.0,0.0,7.8292,0,1,1,1,0
1,47.0,1.0,0.0,7.0000,0,1,0,0,1
2,62.0,0.0,0.0,9.6875,1,0,1,1,0
3,27.0,0.0,0.0,8.6625,0,1,1,0,1
4,22.0,1.0,1.0,12.2875,0,1,0,0,1


In [ ]:
dfMinMax2 = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df_test),columns=df_test.columns, index=df_test.index)
dfMinMax2.head()

,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0.452723,0.000,0.000000,0.015282,0.0,1.0,1.0,1.0,0.0
1,0.617566,0.125,0.000000,0.013663,0.0,1.0,0.0,0.0,1.0
2,0.815377,0.000,0.000000,0.018909,1.0,0.0,1.0,1.0,0.0
3,0.353818,0.000,0.000000,0.016908,0.0,1.0,1.0,0.0,1.0
4,0.287881,0.125,0.111111,0.023984,0.0,1.0,0.0,0.0,1.0


In [ ]:
# put the columns in the same order as the dataframe that was used to train the data
dfMinMax2 = dfMinMax2[dfMinMax.drop(columns='Survived_1').columns]

dfMinMax2.head()
prediction = model_log.predict(dfMinMax2)
predictions = {'PassengerId':passId,'Survived':prediction}
dfPredict = pd.DataFrame(predictions,dtype=int)

In [ ]:
dfPredict.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [ ]:
dfPredict.dtypes

PassengerId    int32
Survived       int32
dtype: object

In [ ]:
dfPredict.to_csv('predictions.csv',index=False)
dfPredict.shape

(418, 2)

In [ ]:
for i in range(2,7):
    print(i)

2
3
4
5
6


In [ ]:
dfPredict.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
